In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import MinMaxScaler

In [3]:
#sigmoid fonk f(x)=1/(1+exp^-x)
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [4]:
def deriv_sigmoid(x):
    return(x*(1-x))

In [5]:
def relu(x):
    return np.maximum(0,x)

In [6]:
def deriv_relu(x):
    der=list()
    der.clear()
    for val in x:
        if val>0:
            der.append(1)
        else:
            der.append(0)
    return der

In [7]:
def leaky_relu(x):
    return np.maximum(0.1*x,x)
    

In [8]:
def deriv_leaky_relu(x):
    der=list()
    der.clear()
    for val in x:
        if val>0:
            der.append(1)
        else:
            der.append(0.1)
    return der

In [9]:
def initialize(x, neuron_num):
    np.random.seed(12345)
    w1=2*np.random.random((len(x.columns), neuron_num))-1
    w2=2*np.random.random((neuron_num, 1))-1
    bias1=2*np.random.random()-1
    bias2=2*np.random.random()-1
    return w1, w2, bias1, bias2

In [10]:
def scaler(data):

    flag_min=1000000
    flag_max=-1000000
    data= data.to_numpy()
    
    for item in data:
        flag_min=np.minimum(item,flag_min)
        flag_max=np.maximum(item,flag_max)
        
    range_=flag_max-flag_min
    new_data=pd.DataFrame()
    for item in data:
        item=(item-flag_min)/range_
    return data
                  

In [11]:
def forward_prop(w1, w2, bias1, bias2, x):
    x=x.reshape(13,1)
    z1=np.dot(w1.T,x)+bias1
    out1=relu(z1)
    #out1=leaky_relu(z1)
    z2=np.dot(w2.T,out1)+bias2
    out2=relu(z2) 
    #out2=leaky_relu(z2)
    return z1, out1, z2, out2

In [12]:
def back_prop(w1, w2, bias1, bias2, x, y, z1, out1, z2, out2, lr, j):
    x=x.reshape(13,1)
    err_2=2*(np.subtract(out2.sum(axis=0), y)) # karenin turevi oldugu icin 2*(y_pred-y_act)
    dummy_2=np.array(deriv_relu(z2)).reshape(1,1)
    #dummy_2=np.array(deriv_leaky_relu(z2)).reshape(1,1)
    dc_dw2=lr*np.dot(out1,dummy_2)*err_2
    w2=np.subtract(w2,dc_dw2)
    dc_db2=lr*err_2
    bias2-=dc_db2

    err_1= 2*(np.subtract(out1.sum(axis=0), y))
    dummy_1=np.array(deriv_relu(z1)).reshape(len(z1),1)
    #dummy_1=np.array(deriv_leaky_relu(z1)).reshape(len(z1),1)
    dc_dw1=lr*np.dot(x,dummy_1.T)*err_1
    w1=np.subtract(w1,dc_dw1)
    dc_db1=lr*err_1
    bias1-=dc_db1
    return w1, w2, bias1, bias2

In [13]:
def forward_prop_test(w1, w2, bias1, bias2, x):
    x=x.reshape(13,1)
    z1=np.dot(w1.T,x)+bias1
    out1=relu(z1)
    z2=np.dot(w2.T,out1)+bias2
    out2=relu(z2) 
    return out1

In [14]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [15]:
columns=("CRIM","ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV")

In [16]:
ds=pd.read_csv("/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/housing.csv", sep='\s+', engine='python', names=columns)

In [17]:
#ds.head()

In [18]:
#ds.isnull().sum()

In [19]:
#ds.describe(include='all')

In [20]:
target=ds.loc[:,'MEDV'].copy()

In [21]:
data=ds.drop('MEDV', axis=1).copy()

In [22]:
neuron_num=10

In [23]:
w1, w2, bias1, bias2=initialize(data,neuron_num)

In [24]:
data_normalized = ((data - data.mean()) / data.std(ddof=0)).to_numpy()
target=target.to_numpy()

In [25]:
cutoff = int(len(data) * 0.8)
x_train, x_test = data_normalized[:cutoff], data_normalized[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [26]:
epochs=100

In [27]:
lr=0.00001

In [28]:
output1=list()
output2=list()
z_1=list()
z_2=list()
loss=0
loss_test=0

In [29]:
for e in range(epochs):
    output1.clear()
    output2.clear()
    z_1.clear()
    z_2.clear()
    errors=0
    for i in range(len(y_train)):   
        z1, out1, z2, out2= forward_prop(w1, w2, bias1, bias2, x_train[i])
        output1.append(out1)
        output2.append(out2)
       # predicted[i]=output[i].sum(axis=0)
        w1, w2, bias1, bias2 = back_prop(w1, w2, bias1, bias2, x_train[i], y_train[i], z1, output1[i], z2, output2[i], lr, i+1)

In [30]:
for i in range(len(y_train)):
   # print(y_train[i], output[i].sum(axis=0))
    loss+=(y_train[i]-(output2[i].sum(axis=0)))

In [31]:
print(loss/(len(y_train)))

[-0.07177979]


In [32]:
out_test=list()
for i in range(len(y_test)): 
    #z_test1, out_test1, z_test2, out_test2=forward_prop(w1, w2, bias1, bias2, x_test[i])
    #out_test.append(out_test2)
    out_test.append(forward_prop_test(w1, w2, bias1, bias2, x_test[i]))
    loss_test+=(y_test[i]-(out_test[i].sum(axis=0)))
print(loss_test/(len(y_test)))

[2.42960878]


In [33]:
for i in range(len(y_test)):
    print(y_test[i], out_test[i].sum(axis=0))

8.5 [8.21754903]
5.0 [11.7864919]
11.9 [15.04443446]
27.9 [18.00221177]
17.2 [11.37242908]
27.5 [10.61804946]
15.0 [9.89723313]
17.2 [7.62348315]
17.9 [9.61374914]
16.3 [10.619848]
7.0 [7.8185717]
7.2 [5.72465527]
7.5 [6.47607015]
10.4 [7.74486867]
8.8 [6.25062746]
8.4 [7.2918171]
16.7 [15.94224062]
14.2 [15.53613414]
20.8 [14.53923171]
13.4 [6.50886312]
11.7 [7.96015843]
8.3 [6.42640137]
10.2 [7.23487829]
10.9 [7.5595827]
11.0 [8.00055751]
9.5 [7.67171951]
14.5 [8.99241697]
14.1 [8.21122099]
16.1 [10.62015522]
14.3 [10.16887735]
11.7 [10.53062294]
13.4 [8.3293765]
9.6 [8.4591508]
8.7 [6.58185474]
8.4 [7.41905158]
12.8 [13.54669806]
10.5 [12.27808582]
17.1 [14.76383831]
18.4 [18.15640012]
15.4 [15.66622139]
10.8 [10.17150427]
11.8 [7.54936397]
14.9 [14.42134485]
12.6 [16.53206097]
14.1 [15.732172]
13.0 [13.17240412]
13.4 [9.05255469]
15.2 [15.8068665]
16.1 [16.14884469]
17.8 [17.17993642]
14.9 [8.46846422]
14.1 [8.74013393]
12.7 [8.30465626]
13.5 [7.95101375]
14.9 [12.10548269]
20.0 [1

In [34]:
#for i in range(len(y_train)):
#    print(y_train[i], output2[i])